<a href="https://colab.research.google.com/github/gusanitor8/Lab2-responsibleAI/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ================================================
# Laboratorio 2 — Similitud de Coseno
# ================================================

# Instrucciones
1. **Carga de embeddings**  
   - Utilizar las representaciones vectoriales preentrenadas (*GloVe 100d*).  
   - Verificar que los términos requeridos estén contenidos en el vocabulario.  

2. **Cálculo de vectores de expresión**  
   - Construir expresiones vectoriales simples mediante operaciones aritméticas (suma, resta).  

3. **Medición de similitud**  
   - Calcular la similitud de coseno entre los vectores de la expresión y los vectores de referencia.  
   - Reportar:  
     - Valor de la similitud de coseno.  
     - Normas de cada vector (\(\|\mathbf{u}\|\)).  

4. **Casos de estudio**  
   Realizar caso por caso

5. **Documentación de resultados**  
   - Registrar en el notebook los valores obtenidos de similitud y normas vectoriales para cada caso.  
   - Incluir comentarios breves interpretando los resultados.  

6. **Conclusiones**  
   - Redactar conclusiones **caso por caso** sobre la utilidad de la similitud de coseno.

In [ ]:
# !pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [ ]:
import numpy as np
from typing import List
import gensim.downloader as api

In [ ]:
# ========= 1) Cargar GloVe 100D =========
model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# ========= 2) Funciones utilitarias =========
def embedding(word: str) -> np.ndarray:
    """Vector de la palabra (lanza KeyError si no está en vocab)."""
    return model[word]

def build_vector(plus: List[str], minus: List[str]) -> np.ndarray:
    """Suma y resta embeddings explícitamente."""
    dim = model[next(iter(model.key_to_index))].shape[0]
    v = np.zeros(dim, dtype=np.float32)
    for w in plus:
        if w in model:
            v += model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    for w in minus:
        if w in model:
            v -= model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    return v

def cos(a: np.ndarray, b: np.ndarray) -> float:
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0 or nb == 0:
        return float("nan")
    return float(np.dot(a, b) / (na * nb))

def vec_norm(v: np.ndarray) -> float:
    return float(np.linalg.norm(v))

def report_case(label, plus, minus, target_plus):
    expr_vec   = build_vector(plus, minus)
    target_vec = build_vector(target_plus, [])
    print("====", label, "====")
    print("Expresión =", " + ".join(plus) + " - " + " - ".join(minus) if minus else " + ".join(plus))
    print("Target    =", " + ".join(target_plus))
    print("cos(expr, target) =", round(cos(expr_vec, target_vec), 6))
    print("||expr||           =", round(vec_norm(expr_vec), 6))
    print("||target||         =", round(vec_norm(target_vec), 6))
    print()

def nearest_neighbors(plus: List[str], minus: List[str], topn: int = 3):
    """Encuentra los vecinos más cercanos a una expresión."""
    expr_vec = build_vector(plus, minus)
    results = model.similar_by_vector(expr_vec, topn=topn)
    print("Vecinos más cercanos para:", " + ".join(plus) + (" - " + " - ".join(minus) if minus else ""))
    for word, score in results:
        print(f"{word:15s} cos = {round(score,6)}")
    print()
    return results

## Caso 1 — Capitales

Construya un vector que represente la relación:  
**paris + italy - france ≈ rome**

1. Defina el vector de la expresión (`expr_vec`).  
2. Defina el vector objetivo (`target_vec`).  
3. Calcule `cos(expr, target)`, `||expr||` y `||target||`.  
4. Escriba su conclusión.  

In [ ]:
# Caso 1 — Capitales
report_case("Caso 1 — Capitales", ["paris","italy"], ["france"], ["rome"])

# Caso 2 — Dictador/País
print("==== Caso 2 — Dictador/País ====")
nearest_neighbors(["hitler","italy"], ["germany"])

# Caso 3 — Religión/Líder
print("==== Caso 3 — Religión/Líder ====")
nearest_neighbors(["christianity"], [])

# Caso 4 — Opuestos
print("==== Caso 4 — Opuestos ====")
nearest_neighbors(["good"], ["bad"])

# Caso 5 — Propio (ejemplo clásico)
print("==== Caso 5 — Propio (king - man + woman) ====")
nearest_neighbors(["king","woman"], ["man"])

==== Caso 1 — Capitales ====
Expresión = paris + italy - france
Target    = rome
cos(expr, target) = 0.8084
||expr||           = 6.227801
||target||         = 5.523363

==== Caso 2 — Dictador/País ====
Vecinos más cercanos para: hitler + italy - germany
mussolini       cos = 0.816139
hitler          cos = 0.715457
fascist         cos = 0.66878

==== Caso 3 — Religión/Líder ====
Vecinos más cercanos para: christianity
christianity    cos = 1.0
catholicism     cos = 0.875112
protestantism   cos = 0.811677

==== Caso 4 — Opuestos ====
Vecinos más cercanos para: good - bad
excellent       cos = 0.522756
versatile       cos = 0.459313
ideal           cos = 0.457311

==== Caso 5 — Propio (king - man + woman) ====
Vecinos más cercanos para: king + woman - man
king            cos = 0.855184
queen           cos = 0.783441
monarch         cos = 0.69338



[('king', 0.8551837205886841),
 ('queen', 0.7834413647651672),
 ('monarch', 0.6933801770210266)]

# Laboratorio 2 — Similitud de Coseno  

## Resultados y Conclusiones Caso por Caso  

### Caso 1 — Capitales  
- **Expresión:** `paris + italy - france`  
- **Target:** `rome`  
- **Similitud coseno:** 0.8084  
- **Normas:**  
  - ||expr|| = 6.2278  
  - ||target|| = 5.5234  

**Interpretación:** El vector resultante se aproxima a *rome*, confirmando que los embeddings capturan la relación capital–país.  

**Conclusión:** La similitud de coseno es útil para representar analogías geográficas y encontrar relaciones entre ciudades y países.  

---

### Caso 2 — Dictador/País  
- **Expresión:** `hitler + italy - germany`  
- **Vecinos más cercanos:**  
  1. mussolini (0.8161)  
  2. hitler (0.7155)  
  3. fascist (0.6688)  

**Interpretación:** *Mussolini* aparece como el vecino más cercano, reflejando una asociación histórica coherente.  

**Conclusión:** La similitud de coseno permite descubrir relaciones políticas e históricas, mostrando su utilidad en contextos semánticos más complejos.  

---

### Caso 3 — Religión/Líder  
- **Expresión:** `christianity`  
- **Vecinos más cercanos:**  
  1. christianity (1.0)  
  2. catholicism (0.8751)  
  3. protestantism (0.8117)  

**Interpretación:** El modelo aproxima *christianity* a religiones afines en lugar de a *jesus*.  

**Conclusión:** La similitud de coseno es útil para medir cercanía semántica entre instituciones religiosas, aunque no siempre captura la relación con líderes individuales.  

---

### Caso 4 — Opuestos  
- **Expresión:** `good - bad`  
- **Vecinos más cercanos:**  
  1. excellent (0.5228)  
  2. versatile (0.4593)  
  3. ideal (0.4573)  

**Interpretación:** El resultado se inclina hacia términos positivos relacionados con *good*, más que hacia un contraste con *bad*.  

**Conclusión:** La similitud de coseno es eficaz para medir semejanza semántica, pero limitada para capturar oposiciones directas.  

---

### Caso 5 — Propio (Analogía género)  
- **Expresión:** `king - man + woman`  
- **Vecinos más cercanos:**  
  1. king (0.8552)  
  2. queen (0.7834)  
  3. monarch (0.6934)  

**Interpretación:** *queen* aparece entre los vecinos más cercanos, confirmando la hipótesis clásica de analogías en embeddings.  

**Conclusión:** La similitud de coseno es muy útil para capturar relaciones semánticas complejas, como las analogías de género en títulos reales.  
